In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import seaborn as sns
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest

In [2]:
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)

In [3]:
train_str = 'data_interpolated_means.csv'
labels_str = 'labels_sorted.csv'
test_str = 'test_features.csv'

df_train = pd.read_csv(train_str)
df_labels = pd.read_csv(labels_str)
df_test = pd.read_csv(test_str)

In [4]:
print(df_train.shape)
df = df_train.copy()

(227940, 37)


In [5]:
df_labels = df_labels.sort_values(by=['pid'])

TEST1 = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate',
         'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2']
TEST2 = ['LABEL_Sepsis']
TEST3 = ['LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']

df_subtask1 = df_labels[TEST1]
df_subtask2 = df_labels[TEST2]
df_subtask3 = df_labels[TEST3]

In [6]:
df_np = df.copy().to_numpy()

In [7]:
df_np = df.copy().to_numpy()

pids = np.unique(df_np[:,0]).astype(int)

# get rid of pid and Time
df_np = df_np[:,2:]

# get the ages
age = df_np[::12,0]
age = age.reshape((len(age),1))

# get rid of the ages 
df_np = df_np[:,1:]

# redefine the matix -> all time data is in one line
df_np = df_np.reshape((len(pids),12*df_np.shape[-1]))

#add the age back
df_np = np.hstack((age,df_np))

#print(df_np.shape)
#df_np[0]

In [8]:
X = df_np
y = df_subtask1['LABEL_BaseExcess'].to_numpy(dtype=float)

print(X.shape)
print(y.shape)

(18995, 409)
(18995,)


In [9]:
clf = LinearDiscriminantAnalysis()
clf.fit(X, y)

LinearDiscriminantAnalysis()

In [10]:
print(clf.predict([[-0.8, -1]]))

ValueError: X has 2 features per sample; expecting 409

In [16]:
df_try = df.copy()
# PREPARING TRAINING FEATURES
df_try = df_try.drop(columns=['Time'])

# fill NaN with median for each column
df_try=(df_try-df_try.mean())/df_try.std()

# convert to numpy array
X = df_try.to_numpy()
pids = np.unique(X[:,0])

# exclude pids
X = X[:,1:]
age = X[::12,0]
age = age.reshape((len(age),1))

# exclude ages
X = X[:,1:]
X = X.reshape((len(pids),12*X.shape[-1]))
X = np.hstack((age,X))

#poly = PolynomialFeatures(10)

# PREPARING TRAINING LABELS

y = df_subtask1.copy().to_numpy()
y = y[:,1:]

#accuracy = np.empty((y.shape[1],))

clf = SGDClassifier()
for i in range(y.shape[1]):
    print("label: " + TEST1[i])
    X_new = SelectKBest(k=409).fit_transform(X, y[:,i])
    #X_new = poly.fit_transform(X_new)
    scores = cross_val_score(clf, X_new, y[:,i], cv=10)
    print(scores)

#for i in range(y.shape[1]):
    #clf = LogisticRegressionCV(cv=10, max_iter=100, solver='sag').fit(X,y[:,i])
    #accuracy[i] = clf.score(X,y)

label: LABEL_BaseExcess
[0.91526316 0.92157895 0.92105263 0.92263158 0.87736842 0.90152712
 0.91943128 0.90521327 0.8957346  0.91416535]
label: LABEL_Fibrinogen
[0.74473684 0.72947368 0.73263158 0.73684211 0.73210526 0.71563981
 0.73986309 0.74828857 0.74776198 0.72985782]
label: LABEL_AST
[0.73       0.71894737 0.72947368 0.73052632 0.73736842 0.76671933
 0.71774618 0.74144286 0.74460242 0.73301738]
label: LABEL_Alkalinephos
[0.73736842 0.72421053 0.75684211 0.73894737 0.72789474 0.7314376
 0.7161664  0.72880463 0.72617167 0.73512375]
label: LABEL_Bilirubin_total
[0.80789474 0.78894737 0.76421053 0.78263158 0.77789474 0.77356503
 0.77567141 0.75566087 0.76724592 0.80252765]
label: LABEL_Lactate
[0.87473684 0.85421053 0.86       0.87157895 0.88315789 0.87993681
 0.87256451 0.88256977 0.87730384 0.90258031]
label: LABEL_TroponinI
[0.78578947 0.75894737 0.76052632 0.76105263 0.78789474 0.73512375
 0.7535545  0.75302791 0.77303844 0.71248025]
label: LABEL_SaO2
[0.95421053 0.96105263 0.956

In [13]:
X.shape

(18995, 409)

In [18]:
y.shape

(18995, 9)